# Mode decomposition of low symmetry structure
### Prepared by Kishwar-E Hasin
(Please suggest corrections to improve the script)

Necessary input files:
1. High symmetry (HS) poscar file
2. Low symmetry (POSCAR) poscar file 
3. Excel file with irrep matrix with given format (need to prepare this file according to the structure)

Stepts:
1. Read HS direct position matrix
2. Read LS direct position matrix
3. Check if any LS matrix elements not greater than certain value (correction-1)
4. Calculate diaplacement matrix
5. Check if the sum of the displacement in each column is 0. If not zero do the correction.(correction-2)
6. Read lattice parameters from HS structure
7. Multiply displacement matrix by associated lattice pvector to convert reduced coordinate to angstrom and convert to vector
8. If any of x or y element present with z element in the matrix, Muptiply the irrep matrix by cell_correction (e.g. sqrt2, sqrt2, 1.) (correction-3)
9. Normalize the irrep matrix 
10. Convert/reshape the irrep matrix to a vector 
11. Find dot product of displacement vector and irrep vector.

In [3]:
import pandas as pd
import numpy as np
import math
from itertools import islice

### These 3 files are needed for the distortion amplitude caculation
high_symmetry_file_name = "HS"
low_symmetry_file_name = "POSCAR"
matrix_file_name= "irrep_matrices_regular_RP_20_atom_cell.xlsx"

cell_correction = np.array([np.sqrt(2.), np.sqrt(2.), 1])            # change according to your computational unit cell
DISPLACEMENT_THRESHOLD = 0.92
PRINT_THRESHOLD = 0.0001

#=========================Change the above names and parameters if needed ====================

## Open and read high symmetry POSCAR file 
with open(high_symmetry_file_name) as lines:         
    lattice_parameter = np.genfromtxt(islice(lines, 2, 5))            # Extract lattice parameter     
        
    for i, line in enumerate(lines):                                  # Count the total number of atoms
        if i == 1:  
            n_atom = sum(map(int, line.strip().split()))      
            break
    high_symmetry = np.genfromtxt(islice(lines, 1, 1 + n_atom))       # Extract the atomic positions from high symmetry structure
    
## Open and read low symmetry POSCAR file 
with open(low_symmetry_file_name) as lines:          
    low_symmetry = np.genfromtxt(islice(lines, 8, 8 + n_atom))        # Extract the atomic positions from high symmetry structure        

displacement_matrix = low_symmetry - high_symmetry                    # Calculate displacement
displacement_matrix = np.where(displacement_matrix > DISPLACEMENT_THRESHOLD, displacement_matrix - 1, displacement_matrix)
displacement_matrix = np.where(displacement_matrix < -DISPLACEMENT_THRESHOLD, displacement_matrix + 1, displacement_matrix)
    
sum_ = np.sum(displacement_matrix, axis=0 )                           # Correction 2: displacement correction 
if np.any(sum_ != 0):  
    correction = sum_ / n_atom
    displacement_matrix = displacement_matrix - correction
    #check = np.sum(displacement_matrix, axis=0) 
    #print (check)   
    displacement_matrix_angstrom = displacement_matrix * np.diag(lattice_parameter)       # Displacement matrix in Angstrom
    displacement_vector = np.reshape(displacement_matrix_angstrom, -1)                    # Convert displacement matrix in vector
    
    
##Define a function that calculate the amplitude "displacement" of an element for a mode/irrep
def calculate_amplitude(irrep_data, element_name):
    df_element = pd.DataFrame(irrep_data[0:n_atom], columns=[f"x_{element_name}", f"y_{element_name}", f"z_{element_name}"])       
    z_nonzero = np.any(df_element[f"z_{element_name}"] != 0)
    xy_nonzero = np.any((df_element[f"x_{element_name}"] != 0) | (df_element[f"y_{element_name}"] != 0))    
    element_matrix = np.array(df_element)
    if z_nonzero and xy_nonzero:
        element_matrix = np.array(df_element) * cell_correction
    norm_element_matrix = np.linalg.norm(element_matrix)           # Calculate the normalization factor
    normalized_element = element_matrix / norm_element_matrix      # Normalization of the matrix
    element_vector = np.reshape(normalized_element, -1)            # Convert matrix to vector     
    displacement = np.dot(displacement_vector, element_vector)     # Displacement amplitude for each element: Dot product     
    return displacement                       

        
### Define a function that calculate the distortion amplitude for all the modes/irreps added in different sheet in the excel file
def process_data(sheet_name, mode_name, element_names):
    ##Calculate the total displacement amplitude of a single mode/irrep 
    irrep_data = pd.read_excel(matrix_file_name, sheet_name, skiprows=1)            # Read the data of irrep matrices        
    for element_name in element_names:
        displacement = calculate_amplitude(irrep_data[0:n_atom], element_name)      # Amplitude "d_element" of all elements for a mode/irrep
        if abs(displacement) > PRINT_THRESHOLD:
            print(f"{element_name}_{mode_name} =", displacement)
        displacement_s = [calculate_amplitude(irrep_data[0:n_atom], element_name) for element_name in element_names]
        mode_name_total = np.sqrt(sum([displacement ** 2 for displacement in displacement_s]))    ## Total amplitude of a mode displacement
    print()
    if mode_name_total > PRINT_THRESHOLD:
        print(f"Total {mode_name} =", mode_name_total)
        print("-----------------------------------")
        
###================ Need to write/change 2 line codes for each Mode to be calculated ===================
# Calculate M2+
element_names_m2 = [ "O"]
process_data('M2+', 'm2', element_names_m2)

# Calculate M3+
element_names_m3 =  ["O"]
process_data('M3+', 'm3', element_names_m3)

# Calculate R4-
element_names_r4 = ["A", "O"]
process_data('R4-', 'r4', element_names_r4)

# Calculate R5-
element_names_r5 = ["O"]
process_data('R5-', 'r5', element_names_r5)

# Calculate X5-
element_names_x5 = ["A", "O"]
process_data('X5-', 'x5', element_names_x5)


O_m2 = 0.7253322265235937

Total m2 = 0.7253322265235937
-----------------------------------
O_m3 = 0.0009693687057283906

Total m3 = 0.0009693687057283906
-----------------------------------
A_r4 = -0.07083928604310244
O_r4 = 0.031311988001380436

Total r4 = 0.07745092019914987
-----------------------------------
O_r5 = 1.1341721859781582

Total r5 = 1.1341721859781582
-----------------------------------
A_x5 = 0.3183644307430506
O_x5 = -0.17958462947040466

Total x5 = 0.36552229741066306
-----------------------------------
